# Prepare the complete data set

This notebook integrates the contents of the urls from "keywords_emptyText.csv" into the data set with all urls

Verizon, Group 41
<br>Athena Bai, Tia Zheng, Kathy Yang, Tapuwa Kabaira, Chris Smith

Last updated: Nov. 28, 2024

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

In [2]:
# Read data files
working_urls = pd.read_csv("df_text.csv", header=0)
remaining_urls = pd.read_csv("remaining_contents.csv", header=0)
labels = pd.read_csv("categorizedurls.csv", header=0)

In [5]:
working_urls.columns.values

array(['url', ' category', 'text_content', 'Text_Length', 'text_cleaned',
       'Sentiment'], dtype=object)

In [7]:
remaining_urls.columns.values

array(['url', 'content'], dtype=object)

In [ ]:
# Make a copy of each DataFrame
working_urls = working_urls.copy()
remaining_urls = remaining_urls.copy()
labels = labels.copy()

In [4]:
from deep_translator import GoogleTranslator
to_translate = '你好，世界！'
translated = GoogleTranslator(source='zh-CN', target='en').translate(to_translate)
print(translated)

Hello World!


In [ ]:
# Add features to the remaining urls
remaining_urls['content'] = remaining_urls['text_content'].str.len().fillna(0)
df['text_cleaned'] = df['text_content'].apply(preprocess_text)
df['Sentiment'] = df['text_cleaned'].apply(calc_sentiment)
df['lexical_diversity'] = df['text_content'].apply(lexical_diversity)

In [ ]:
# Prepare df2 to match df1's structure with None for missing columns
df2_prepared = df2[['url']].copy()
df2_prepared['category'] = None
df2_prepared['text_content'] = None
df2_prepared['Text_Length'] = None
df2_prepared['text_cleaned'] = None
df2_prepared['Sentiment'] = None

# Concatenate df1 and prepared df2
df_combined = pd.concat([df1, df2_prepared], ignore_index=True)

# Merge with df3 to ensure the correct order based on 'url' and fill the 'category' column
df4 = df3[['url', 'category']].merge(
    df_combined, on=['url', 'category'], how='left'
)